In [202]:
import numpy as np
import pandas as pd
import random


In [203]:
df = pd.read_csv("data-raw.csv")
df["Status Start Date"] = pd.to_datetime(df["Status Start Date"])
df["Status End Date"] = pd.to_datetime(df["Status End Date"])


In [ ]:
def update_revenue_profit(x,column_name):
    if(x["Sales Stage"] != "Closed"):
        return x[column_name] * np.random.uniform(96,100)        
    else:
        return x[column_name]
    
def update_end_date(x):
    if(x["Sales Stage"] != "Closed"):
        return x["Status End Date"]+pd.DateOffset(np.random.randint(5,50))
    else:
        return x["Status End Date"]


def create_dataset_by_status_percentage(df,status,percentage):    
    df = df.loc[df["Sales Stage"] != "Closed"] #Get rows where status is not Closed
    df.index = range(len(df.index))
    df["Sales Stage"] = "Closed" #First Set All rows to Closed
    rand_index = random.sample(range(0,df.shape[0]),int(percentage*float(df.shape[0]))) #randomly select rows not to mark closed
    df.loc[rand_index,"Sales Stage"] = status  #set the status of random indexes to "Status"
    df.loc[:,"Status Start Date"] = df.loc[:,"Status End Date"] #Set Start Date to End Date of Previous status
    df.apply(lambda x:update_end_date(x),axis=1) #Update End Date to new date when status is not closed
    df.apply(lambda x:update_revenue_profit(x,"Revenue"),axis=1) #Update Revenue when status is not closed
    df.apply(lambda x:update_revenue_profit(x,"Net Revenue"),axis=1) #Update Net Revnue when status is not closed
    df.index = range(len(df.index))
    return df;



In [ ]:
q = create_dataset_by_status_percentage(df,"Qualify",np.random.uniform(0.65,0.85))
df = df.append(q)
s = create_dataset_by_status_percentage(q,"Solution",np.random.uniform(0.55,0.75))
df = df.append(s)
p = create_dataset_by_status_percentage(s,"Proposal",np.random.uniform(0.60,0.87))
df = df.append(p)
d = create_dataset_by_status_percentage(p,"Deal",np.random.uniform(0.78,0.95))
df = df.append(d)
df.to_csv("opportunity_data_complete.csv")


In [ ]:
d.shape